In [3]:
import collections
import torch
from torch import cat, no_grad, manual_seed
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import tqdm
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from QuICT_ml.ansatz_library import QNNLayer
from QuICT_ml.utils.encoding import *
from QuICT_ml.utils.ml_utils import *
from QuICT_ml.model.QNN import QuantumNet

In [27]:
np.random.seed = 42
manual_seed(42)
EPOCH = 50       # 训练总轮数
BATCH_SIZE = 64 # 一次迭代使用的样本数
LR = 0.001      # 梯度下降的学习率
SEED = 42       # 随机数种子

In [12]:
X_train = datasets.MNIST(root="./data/", train=True, download=True)
batch_size = 64
n_samples = 1024  # We will concentrate on the first 100 samples
# 创建一个索引列表，包含所有类别（0-9）的样本
idx = []
for label in range(2):  # 遍历所有类别（0-9）
    label_idx = np.where(X_train.targets == label)[0][:n_samples]  # 获取当前类别的样本索引
    idx.append(label_idx)

# 将所有类别的索引合并为一个数组
idx = np.concatenate(idx)

# 根据索引过滤数据
X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]
train_X = X_train.data
train_Y = X_train.targets

n_samples = 512
X_test = datasets.MNIST(root="./data/", train=False, download=True)

idx = []
for label in range(2):  # 遍历所有类别（0-9）
    label_idx = np.where(X_test.targets == label)[0][:n_samples]  # 获取当前类别的样本索引
    idx.append(label_idx)

# 将所有类别的索引合并为一个数组
idx = np.concatenate(idx)

# 根据索引过滤数据
X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]
test_X = X_test.data
test_Y = X_test.targets
print("Training examples: ", len(train_Y))
print("Testing examples: ", len(test_Y))

Training examples:  2048
Testing examples:  1024


In [13]:
def downscale(X, resize):
    transform = transforms.Resize(size=resize)
    X = transform(X) / 255.0
    return X

resized_train_X = downscale(train_X, (4, 4))
resized_test_X = downscale(test_X, (4, 4))

c:\Users\ly\.conda\envs\Qenv\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [14]:
def remove_conflict(X, Y, resize):
    x_dict = collections.defaultdict(set)
    for x, y in zip(X, Y):
        x_dict[tuple(x.numpy().flatten())].add(y.item())
    X_rmcon = []
    Y_rmcon = []
    for x in x_dict.keys():
        if len(x_dict[x]) == 1:
            X_rmcon.append(np.array(x).reshape(resize))
            Y_rmcon.append(list(x_dict[x])[0])
    X = torch.from_numpy(np.array(X_rmcon))
    Y = torch.from_numpy(np.array(Y_rmcon))
    return X, Y

nocon_train_X, nocon_train_Y = remove_conflict(resized_train_X, train_Y, (4, 4))
nocon_test_X, nocon_test_Y = remove_conflict(resized_test_X, test_Y, (4, 4))
print("Remaining training examples: ", len(nocon_train_Y))
print("Remaining testing examples: ", len(nocon_test_Y))


Remaining training examples:  1462
Remaining testing examples:  629


In [15]:
def binary_img(X, threshold):
    X = X > threshold
    X = X.type(torch.int)
    return X

threshold = 0.5
bin_train_X = binary_img(nocon_train_X, threshold)
bin_test_X = binary_img(nocon_test_X, threshold)

In [16]:
device = torch.device("cuda:0")

train_X = bin_train_X.to(device)
train_Y = nocon_train_Y.to(device)
test_X = bin_test_X.to(device)
test_Y = nocon_test_Y.to(device)

In [18]:
def qubit_encoding(X, device):
    new_X = []
    n_qubits = X[0].shape[0] * X[0].shape[1]
    qe = Qubit(n_qubits, device)
    for x in X:
        qe.encoding(x)
        new_X.append(qe.ansatz)
    return new_X

In [19]:
ansatz_train_X = qubit_encoding(train_X, device)
ansatz_test_X = qubit_encoding(test_X, device)

In [20]:
pqc = QNNLayer(list(range(4)), 4, device=device)
params = nn.Parameter(torch.rand(1, 4, device=device), requires_grad=True)
model_circuit = pqc.circuit_layer(["XX"], params)
model_circuit.draw()

<Figure size 848.056x645 with 1 Axes>

In [21]:
data_qubits = list(range(16))
readout_qubit = 16
pqc = QNNLayer(data_qubits, readout_qubit, device=device)
layers = ["XX", "ZZ"]
params = nn.Parameter(torch.rand(2, 16, device=device), requires_grad=True)
model_ansatz = pqc(layers, params)

In [33]:
train_dataset = data.TensorDataset(train_X, train_Y)
test_dataset = data.TensorDataset(test_X, test_Y)
train_loader = data.DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True
)
test_loader = data.DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True
)

In [24]:
net = QuantumNet(16, layers, encoding="qubit", device=device)
optim = torch.optim.Adam([dict(params=net.parameters(), lr=LR)])

In [35]:
def loss_func(y_true, y_pred):
    y_true = 2 * y_true.type(torch.float32) - 1.0
    y_pred = 2 * y_pred - 1.0
    loss = torch.clamp(1 - y_pred * y_true, min=0.0)
    correct = torch.where(y_true * y_pred > 0)[0].shape[0]
    return torch.mean(loss), correct

In [36]:
# train epoch
for ep in range(EPOCH):
    net.train()
    loader = tqdm.tqdm(
        train_loader, desc="Training epoch {}".format(ep + 1), leave=True
    )
    # train iteration
    for it, (x_train, y_train) in enumerate(loader):
        optim.zero_grad()
        y_pred = net(x_train)

        loss, correct = loss_func(y_train, y_pred)
        accuracy = correct / len(y_train)
        loss.backward()
        optim.step()
        loader.set_postfix(
            it=it,
            loss="{:.3f}".format(loss),
            accuracy="{:.3f}".format(accuracy),
        )

    # Validation
    net.eval()
    loader_val = tqdm.tqdm(
        test_loader, desc="Validating epoch {}".format(ep + 1), leave=True
    )
    loss_val_list = []
    total_correct = 0
    for it, (x_test, y_test) in enumerate(loader_val):
        y_pred = net(x_test)
        loss_val, correct = loss_func(y_test, y_pred)
        loss_val_list.append(loss_val.cpu().detach().numpy())
        total_correct += correct
        accuracy_val = correct / len(y_test)
        loader_val.set_postfix(
            it=it,
            loss="{:.3f}".format(loss_val),
            accuracy="{:.3f}".format(accuracy_val),
        )
    avg_loss = np.mean(loss_val_list)
    avg_acc = total_correct / (len(loader_val) * BATCH_SIZE)
    print("Validation Average Loss: {}, Accuracy: {}".format(avg_loss, avg_acc))

Validating epoch 1: 100%|██████████| 9/9 [02:47<00:00, 18.57s/it, accuracy=0.547, it=8, loss=0.950]


Validation Average Loss: 0.9631057381629944, Accuracy: 0.5416666666666666


Validating epoch 2: 100%|██████████| 9/9 [02:47<00:00, 18.59s/it, accuracy=0.594, it=8, loss=0.915]


Validation Average Loss: 0.8825179934501648, Accuracy: 0.6145833333333334


Validating epoch 3: 100%|██████████| 9/9 [02:47<00:00, 18.58s/it, accuracy=0.766, it=8, loss=0.809]


...

Validating epoch 44: 100%|██████████| 9/9 [15:32<00:00, 103.66s/it, accuracy=0.844, it=8, loss=0.443]


Validation Average Loss: 0.533491849899292, Accuracy: 0.7725694444444444


Validating epoch 45: 100%|██████████| 9/9 [15:22<00:00, 102.53s/it, accuracy=0.797, it=8, loss=0.507]


Validation Average Loss: 0.5345349907875061, Accuracy: 0.7743055555555556


Validating epoch 46:  78%|███████▊  | 7/9 [13:04<04:00, 120.18s/it, accuracy=0.859, it=6, loss=0.411]